<a href="https://colab.research.google.com/github/jiin124/Dacon/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EC%A7%80%EC%A0%90%EB%B3%84_%EB%A7%A4%EC%B6%9C%EC%95%A1_%EC%98%88%EC%B8%A1/%EB%8D%B0%EC%9D%B4%EC%BD%98_Basic_%EC%87%BC%ED%95%91%EB%AA%B0_%EC%A7%80%EC%A0%90%EB%B3%84_%EB%A7%A4%EC%B6%9C%EC%95%A1_%EC%98%88%EC%B8%A1%7C_Private_5%EC%9C%84(35096)_%7C_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이콘 Basic 쇼핑몰 지점별 매출액 예측 경진대회
 https://dacon.io/competitions/official/235942/codeshare/5992?page=1&dtype=recent

1. 공휴일 숫자로 처리
2. 날짜 분리 
3. 결측치는 모두 0으로 바꿈
4. 프로모션 변수는 로그 변환
5. store 원핫 인코딩. 
6. 타겟변수 로그 변환 후 필요없는 변수 제거
-> id, date,temperature,fuel_price, promotion1,promotion2,unemployment,promotion4, promotion5, isholiday
7. 랜덤포레스트 사용. 




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
sample=pd.read_csv('/content/sample_submission.csv')

In [ ]:
display(train)
print()
display(test)

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251,45,31/08/2012,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,8.684,False,734297.87
6251,6252,45,07/09/2012,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66
6252,6253,45,14/09/2012,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27
6253,6254,45,21/09/2012,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20


,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday
0,1,1,05/10/2012,68.55,3.617,8077.89,NaN,18.22,3617.43,3626.14,6.573,False
1,2,1,12/10/2012,62.99,3.601,2086.18,NaN,8.11,602.36,5926.45,6.573,False
2,3,1,19/10/2012,67.97,3.594,950.33,NaN,4.93,80.25,2312.85,6.573,False
3,4,1,26/10/2012,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,False
4,5,2,05/10/2012,70.27,3.617,6037.76,NaN,10.04,3027.37,3853.40,6.170,False
...,...,...,...,...,...,...,...,...,...,...,...,...
175,176,44,26/10/2012,46.97,3.755,38.65,2.61,0.98,NaN,457.74,5.217,False
176,177,45,05/10/2012,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,8.667,False
177,178,45,12/10/2012,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,8.667,False
178,179,45,19/10/2012,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,8.667,False


## 전처리

In [3]:
train.loc[train['IsHoliday']==True,'IsHoliday']=1
test.loc[test['IsHoliday']==False,'IsHoliday']=0

In [4]:
test.loc[test['IsHoliday']==True,'IsHoliday']=1
train.loc[train['IsHoliday']==False,'IsHoliday']=0

In [6]:
train=train.astype({'IsHoliday':'int64'})#형식 바꾸어주기
test=test.astype({'IsHoliday':'int64'})#형식 바꾸어주기

In [7]:
#datetime
train['Date']=pd.to_datetime(train['Date'])

In [8]:
import datetime as dt

train['Year']=train['Date'].dt.year
train['Month']=train['Date'].dt.month
train['Day']=train['Date'].dt.day

In [9]:
train

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,Year,Month,Day
0,1,1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,0,1643690.90,2010,5,2
1,2,1,2010-12-02,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,1,1641957.44,2010,12,2
2,3,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,0,1611968.17,2010,2,19
3,4,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,0,1409727.59,2010,2,26
4,5,1,2010-05-03,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,0,1554806.68,2010,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251,45,2012-08-31,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,8.684,0,734297.87,2012,8,31
6251,6252,45,2012-07-09,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,1,766512.66,2012,7,9
6252,6253,45,2012-09-14,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,0,702238.27,2012,9,14
6253,6254,45,2012-09-21,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,0,723086.20,2012,9,21


In [10]:
test['Date']=pd.to_datetime(test['Date'])
test['Year']=test['Date'].dt.year
test['Month']=test['Date'].dt.month
test['Day']=test['Date'].dt.day

In [11]:
test['IsHoliday']=test['IsHoliday'].astype('int64')

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            180 non-null    int64         
 1   Store         180 non-null    int64         
 2   Date          180 non-null    datetime64[ns]
 3   Temperature   180 non-null    float64       
 4   Fuel_Price    180 non-null    float64       
 5   Promotion1    178 non-null    float64       
 6   Promotion2    45 non-null     float64       
 7   Promotion3    161 non-null    float64       
 8   Promotion4    146 non-null    float64       
 9   Promotion5    180 non-null    float64       
 10  Unemployment  180 non-null    float64       
 11  IsHoliday     180 non-null    int64         
 12  Year          180 non-null    int64         
 13  Month         180 non-null    int64         
 14  Day           180 non-null    int64         
dtypes: datetime64[ns](1), float64(8), int64(

In [13]:
#결측치 처리
#결측치 모두 0으로 바꾼 후 프로모션 변수는 로그 변환

train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

train['Promotion1'][train['Promotion1']<0]=0
train['Promotion2'][train['Promotion2']<0]=0
train['Promotion3'][train['Promotion3']<0]=0
train['Promotion4'][train['Promotion4']<0]=0
train['Promotion5'][train['Promotion5']<0]=0

test['Promotion1'][test['Promotion1']<0]=0
test['Promotion2'][test['Promotion2']<0]=0
test['Promotion3'][test['Promotion3']<0]=0
test['Promotion4'][test['Promotion4']<0]=0
test['Promotion5'][test['Promotion5']<0]=0

train['Promotion1']=np.log1p(train['Promotion1'])
train['Promotion2']=np.log1p(train['Promotion2'])
train['Promotion3']=np.log1p(train['Promotion3'])
train['Promotion4']=np.log1p(train['Promotion4'])
train['Promotion5']=np.log1p(train['Promotion5'])

test['Promotion1']=np.log1p(test['Promotion1'])
test['Promotion2']=np.log1p(test['Promotion2'])
test['Promotion3']=np.log1p(test['Promotion3'])
test['Promotion4']=np.log1p(test['Promotion4'])
test['Promotion5']=np.log1p(test['Promotion5'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: Setti

In [14]:
#store 원핫인코딩
train=pd.get_dummies(train,columns=['Store'])
test=pd.get_dummies(test,columns=['Store'])
print(train.shape)
print(test.shape)
train.head()

(6255, 60)
(180, 59)


,id,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,1,2010-05-02,42.31,2.572,0.0,0.0,0.0,0.0,0.0,8.106,...,0,0,0,0,0,0,0,0,0,0
1,2,2010-12-02,38.51,2.548,0.0,0.0,0.0,0.0,0.0,8.106,...,0,0,0,0,0,0,0,0,0,0
2,3,2010-02-19,39.93,2.514,0.0,0.0,0.0,0.0,0.0,8.106,...,0,0,0,0,0,0,0,0,0,0
3,4,2010-02-26,46.63,2.561,0.0,0.0,0.0,0.0,0.0,8.106,...,0,0,0,0,0,0,0,0,0,0
4,5,2010-05-03,46.50,2.625,0.0,0.0,0.0,0.0,0.0,8.106,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#타겟 변수 로그 변환 후 필요없는 변수 제거
train['Weekly_Sales']=np.log1p(train['Weekly_Sales'])
train.drop(['id','Date','Temperature','Fuel_Price','Promotion1','Promotion2','Unemployment','Promotion4','Promotion5','IsHoliday'],axis=1,inplace=True)
test.drop(['id','Date','Temperature','Fuel_Price','Promotion1','Promotion2','Unemployment','Promotion4','Promotion5','IsHoliday'],axis=1,inplace=True)

In [16]:
train

,Promotion3,Weekly_Sales,Year,Month,Day,Store_1,Store_2,Store_3,Store_4,Store_5,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,0.000000,14.312455,2010,5,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,14.311400,2010,12,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,14.292967,2010,2,19,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,14.158908,2010,2,26,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,14.256862,2010,5,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,4.542550,13.506671,2012,8,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6251,3.982109,13.549608,2012,7,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6252,1.667707,13.462029,2012,9,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6253,4.162626,13.491285,2012,9,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## 모델링

In [17]:
x_train=train.drop(columns='Weekly_Sales',axis=1)
y_train=train['Weekly_Sales']

In [ ]:
#from sklearn.model_selection import train_test_split

#X_train,X_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.3,random_state=0)
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)

(4378, 49)
(1877, 49)
(4378,)
(1877,)


In [18]:
from sklearn.ensemble import RandomForestRegressor

rf=RandomForestRegressor(random_state=42,n_estimators=200)
rf.fit(x_train,y_train)
pred=np.expm1(rf.predict(test))

In [ ]:
#from sklearn.metrics import mean_squared_error

#def RMSE(y_pred,y):
#  return mean_squared_error(y_pred,y)**0.5

In [ ]:
#rmse1=RMSE(pred,y_test)
#print(rmse1)

1182267.7897011586


In [19]:
sample['Weekly_Sales']=pred
sample.head()

,id,Weekly_Sales
0,1,1.633035e+06
1,2,1.569304e+06
2,3,1.552166e+06
3,4,1.490552e+06
4,5,1.914113e+06
